# COBRApy

COBRApy is a package for constraint-based modeling of biological networks written in Python.

This tool allows loading and inspecting Genome-Scale Metabolic (GEM) models written in the Systems Biology Markup Language (SBML) format.

Using COBRApy, one can analyse the following model contents:
1. Reactions
2. Metabolites
3. Genes
4. Exchange reactions (Environmental Conditions)

COBRApy allows manipulating the contents of a GEM model. For instance, one can edit reactions' flux bounds, knock out a metabolic gene, or change the environmental conditions.

Phenotype prediction can be simulated with several flux analysis methods implemented in COBRApy. These include Flux Balance Analysis (FBA), Parsimonious FBA, or Flux Variability Analysis (FVA).

The simulation of gene and reaction deletions for a given GEM model is a simple and straightforward process. One can simulate single or double knock outs using one of the flux analysis methods mentioned above.

## Instalation


### Requirements
The following requirements need to be installed to use COBRApy:
- Python 3.6 or higher
- pip must be installed
- GLPK solver is used by default but CPLEX is prefered


### How to install COBRApy?
```
pip install cobra
```

## Working with a GEM model

COBRApy can be used to read a GEM model in SBML format file.
For this practical session, we will be using two models:
- _textbook_ model which contains the metabolic network presented in the _exercice 1_ -> file: ../data/textbook.xml

The reactions, metabolites, and genes encoded into a SBML format file can be parsed by COBRApy. These contents are loaded into Python objects simple to use, namely `cobra.Reaction`, `cobra.Metabolite`, and `cobra.Gene`

The model itself will be available as an `cobra.Model` object containing all these attributes.

In [1]:
# importing cobra
import cobra

# Loading a model
model_path = '../data/textbook.xml'
model = cobra.io.read_sbml_model(model_path)

model

Name,textbook
Memory address,0x01bd83108ac8
Number of metabolites,9
Number of reactions,12
Number of groups,0
Objective expression,1.0*R4 - 1.0*R4_reverse_8717c
Compartments,c


### Linear problem
The number of degrees of freedom can be calculated by subtracting the number of variables by the number of linear mass equations. That is, one can determine the degree of under-determination of this model by subtracting the number reactions by the number of metabolites.

In [2]:
# calculing the linear problem degrees of freedom
degrees = len(model.reactions) - len(model.metabolites)
degrees

3

## Environmental Conditions


The set of measured exchange fluxes can be used to define the environmental conditions of our _textbook_ model. Thus, we can use the measuments obtained to turn our under-determined system into a determined system ready to be solved.

In [ ]:
# create the environmental conditions with our measurements
environmental_conditions = {
    'R1': (-1, 0),
    'R2': (0.5, 1000),
    'R3': (0.5, 1000),
}

In [ ]:
# set the environmental conditions in the textbook model
for reaction_id, bound in environmental_conditions.items():
    model.reactions.get_by_id(reaction_id).bounds = bound

### Flux analysis with FBA

COBRApy allows performing flux analysis in GEM models using Flux Balance Analysis (FBA).

To perform a simulation using one of these methods, you should first define an objective function. This can be a reaction or an exchange, which will be maximized or minimized. 

By default, the _textbook_ model objective function is reaction 4, R4. 
This can be thought of as the maximization of the ethanol production.

In [ ]:
model.objective

#### Flux Balance Analysis (FBA)

FBA simulations can be performed using `model.optimize()`. One can easily see the flux distribution as a result.

In [ ]:
flux_distribution = model.optimize()
flux_distribution

### Pathway Visualization

COBRApy does not include any pathway visualization method. Nevertheless, independent tools that work with cobra models can be use for that goal. One of such tools is named Escher.

Escher allows the visualization of metabolic pathway maps. Escher maps can be build using the `escher.Build()` function, which requires the model and a FBA solution.

To obtain the metabolic map for the model in question, Escher can be run as follows:

In [ ]:
# import escher
import escher

In [ ]:
builder = escher.Builder(map_json='../data/textbook_map.json', model=model, reaction_data=flux_distribution.fluxes)
builder